In [61]:
import pandas as pd
import numpy as np
import warnings
import re
import os

warnings.filterwarnings('ignore')

In [62]:
unprocessed_dir = os.path.abspath('../../unprocessed')
processed_dir = os.path.abspath('../../processed')
file = os.path.join(unprocessed_dir,'results.csv')
results = pd.read_csv(file, encoding = 'utf-8', dtype = {'PRECINCT_CODE':str})
results.shape

(20552238, 10)

In [43]:
contest = pd.read_csv(os.path.join(processed_dir, 'static/contests.csv'), encoding = 'utf-8')
# contest.rename({0:'CONTEST_CODE', 1: 'CONTEST_NAME'}, axis = 1, inplace = True)
positions = ['SENATOR', 'GOVERNOR', 'VICE-GOVERNOR', 'MAYOR', 'VICE-MAYOR','PARTY LIST']

In [44]:
codes = []
for pos in positions:
    mask = contest['CONTEST_NAME'].str.contains(pos)
    codes_1 = contest.loc[mask]
    codes.append(codes_1)
codes = pd.concat(codes)

In [45]:
candidates = pd.read_csv(os.path.join(processed_dir, 'static/candidates.csv'), encoding = 'utf-8')
candidates = candidates.loc[candidates['CONTEST_CODE'].isin(codes['CONTEST_CODE'].values)]

In [46]:
precincts = pd.read_csv(os.path.join(processed_dir, 'static/precincts.csv'), encoding = 'utf-8', dtype = {'VCM_ID':str})
precincts.columns

Index(['VCM_ID', 'REG_NAME', 'PRV_NAME', 'MUN_NAME', 'BRGY_NAME', 'POLLPLACE',
       'CLUSTERED_PREC', 'REGISTERED_VOTERS'],
      dtype='object')

In [47]:
results = results.merge(precincts, left_on = 'PRECINCT_CODE', right_on = 'VCM_ID', how = 'left')

In [48]:
results.columns

Index(['PRECINCT_CODE', 'CONTEST_CODE', 'CANDIDATE_NAME', 'PARTY_CODE',
       'VOTES_AMOUNT', 'TOTALIZATION_ORDER', 'NUMBER_VOTERS', 'UNDERVOTE',
       'OVERVOTE', 'RECEPTION_DATE', 'VCM_ID', 'REG_NAME', 'PRV_NAME',
       'MUN_NAME', 'BRGY_NAME', 'POLLPLACE', 'CLUSTERED_PREC',
       'REGISTERED_VOTERS'],
      dtype='object')

In [52]:
results = results.loc[results['REG_NAME'] != 'OAV']
regions = results['REG_NAME'].unique()
unique_candidates = results.CANDIDATE_NAME.unique()

In [54]:
results.set_index('CANDIDATE_NAME', inplace = True)

In [59]:
grouped = results.groupby(['REG_NAME', 'CANDIDATE_NAME'])
keys = grouped.groups.keys()
region_summary = pd.DataFrame(index = results.REG_NAME.unique().tolist(), columns = candidates.CANDIDATE_NAME.values)
for key in keys:
    a = grouped.get_group(key)
    region_summary.loc[key[0]][key[1]] = a.VOTES_AMOUNT.sum()
region_summary
    

,"ALVAREZ, JCA (PDPLBN)","ABUY, BENILDA FATIMA (PFP)","YNARES, NINI (NPC)","JECOY, VILLA (PDPLBN)","ESCUDERO, CHIZ (NPC)","GALINDES, PADS ROLLIN (IND)","BALMEO, BP (IND)","BUENAFLOR, BISHOP (PDDS)","BANAAG, PERIOLO (PFP)","SUPLIG, MARLON (LAKAS)",...,92 AKO BISAYA,31 JUAN MOVEMENT,8 1-UTAK,73 ALIF,127 AWAKE,139 ANUPA,103 APPEND,86 ALAY BUHAY,27 AMEPA OFW,108 ABAKADA
REGION IV-B,312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50,83,77,57,50,52,57,48,81,48
REGION III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,193,213,210,225,174,169,189,213,213,182
REGION IV-A,NaN,621,1423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,226,322,290,229,204,216,211,234,317,201
REGION VIII,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123,126,126,126,108,113,115,122,125,122
CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49,42,42,45,36,42,42,47,42,33
REGION XI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,78,97,104,91,81,72,70,83,100,77
REGION IX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72,87,102,51,66,69,72,60,93,69
REGION XIII,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43,96,99,43,29,36,41,39,99,42
REGION I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120,120,116,114,108,105,113,117,114,108
REGION XII,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,60,94,81,71,63,63,66,63,93,72


In [60]:
grouped = results.groupby(['MUN_NAME', 'CANDIDATE_NAME'])
keys = grouped.groups.keys()
mun_summary = pd.DataFrame(index = results.MUN_NAME.unique().tolist(), columns = candidates.CANDIDATE_NAME.values)
for key in keys:
    a = grouped.get_group(key)
    mun_summary.loc[key[0]][key[1]] = a.VOTES_AMOUNT.sum()
mun_summary

,"ALVAREZ, JCA (PDPLBN)","ABUY, BENILDA FATIMA (PFP)","YNARES, NINI (NPC)","JECOY, VILLA (PDPLBN)","ESCUDERO, CHIZ (NPC)","GALINDES, PADS ROLLIN (IND)","BALMEO, BP (IND)","BUENAFLOR, BISHOP (PDDS)","BANAAG, PERIOLO (PFP)","SUPLIG, MARLON (LAKAS)",...,92 AKO BISAYA,31 JUAN MOVEMENT,8 1-UTAK,73 ALIF,127 AWAKE,139 ANUPA,103 APPEND,86 ALAY BUHAY,27 AMEPA OFW,108 ABAKADA
EL NIDO (BACUIT),18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,3,2,0,0,0,0,0,3,0
LINAPACAN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,0,0,0,0,0,3,0,0,0
MAGSAYSAY,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,1,6,3,2,0,3,0,5,0
NARRA,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,3,3,0,0,0,0,0,3,0
PUERTO PRINCESA CITY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,3,3,3,6,6,3,3,3,6
QUEZON,21,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7,9,0,6,3,3,3,9,6,3
ROXAS,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,5,6,6,0,0,3,3,6,0
SAN VICENTE,12,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,...,3,3,3,0,0,3,0,0,3,0
TAYTAY,27,NaN,133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,6,6,6,3,8,3,3,6,0
KALAYAAN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,3,0
